## 2. Branching & Node Selection方法

本notebook对应[这一篇blog post](https://hanqiu92.github.io/blogs/2020/IP_BB_solver_2_202006/)中的内容，主要包括B&B方法中branching和node selection步骤的一些常用方法的实现。

首先是准备工作：导入相关的计算工具包、从文件util_lec_2.py中读取上一个notebook中完成的B&B框架。

In [1]:
import time
import math
import numpy as np
from cylp.cy import CyClpSimplex
from util import *
from util_lec_2 import *
np.set_printoptions(suppress=True,precision=4)

在此前的B&B框架设计中，我们已经为branching和node selection阶段的实现方法扩展留出了接口：只需要重载类Brancher的branch方法和类Selector的select方法即可。不过，由于部分新方法的执行依赖于一些新引入信息，例如pseudo cost、estimate等，因此我们还需要对原有的B&B求解框架进行扩展。具体来说，下面的代码包括以下三个模块：

* 对原有**Node类**和**BBSolver类**的扩展
* 引入**PseudoCostManager类**，封装pseudo cost的相关计算
* 在**Brancher类**和**Selector类**实现blog中提到的常用方法，并增加相应的类型标签

Node类的扩展包括在update方法中加入一系列（branch和select所需的）中间变量的计算，以及加入estimate的计算。需要注意的是，为了节省计算量，我们将仅在创建结点和处理结点两个阶段对estimate进行计算，而不是每次pseudo cost更新都重新计算。这种处理方式与SCIP中的实现一致。

In [2]:
class NodeExtend(Node):
    def __init__(self,node_idx=0,parent=None,cost_manager=None,
                 var_idx=-1,var_sol=0,var_bnd=0,
                 var_lb=0,var_ub=0,init_var_lb=0,init_var_ub=0):
        '''
        初始化的输入中增加了pseudo cost manager，用于estimate的初始化计算
        '''
        ## 进行基础的初始化
        super().__init__(node_idx=node_idx,parent=parent,
                 var_idx=var_idx,var_sol=var_sol,var_bnd=var_bnd,
                 var_lb=var_lb,var_ub=var_ub,init_var_lb=init_var_lb,init_var_ub=init_var_ub)
        
        ## 进一步初始化estimate
        self.estimate = MAX
        if parent is not None:
            self.estimate = parent.estimate
            if cost_manager is not None:
                ## 根据pseudo cost对estimate做调整
                ## Note：要求parent node保留了中间变量delta_plus和delta_minus
                var_delta = self.var_delta
                cost_plus,cost_minus = cost_manager.get_cost()
                if var_delta > 0:
                    self.estimate = parent.estimate + var_delta * cost_plus[var_idx] ## 向上调整
                else:
                    self.estimate = parent.estimate - var_delta * cost_minus[var_idx] ## 向下调整
                ## 去除变量var_idx原来的贡献值
                self.estimate -= min(parent.delta_plus[var_idx]*cost_plus[var_idx],
                                     parent.delta_minus[var_idx]*cost_minus[var_idx])
            
    def activate(self,solver):
        super().activate(solver)
        ## 增加中间变量
        self.sol_bool_infeas,self.sol_int_infeas,self.sol_bool_feas = None,None,None ## 记录每个变量是否满足整数约束
        self.delta_minus,self.delta_plus = None,None ## 每个变量为满足整数可行性约束所应变化的大小（-、+）

    def update(self,sol,obj,solver):
        ## 保存LP求解结果
        self.sol,self.obj,self.LB = sol,obj,obj
        ## 增加中间变量的计算
        self.sol_bool_infeas = (np.abs(sol - np.round(sol)) > TOL) & solver.int_vars_bool
        self.sol_int_infeas = np.where(self.sol_bool_infeas)[0]
        self.sol_feas = (~np.any(self.sol_bool_infeas))
        self.sol_bool_feas = ~self.sol_bool_infeas
        sol_elem_infeas = sol[self.sol_bool_infeas]
        self.delta_minus,self.delta_plus = np.zeros((len(sol),)),np.zeros((len(sol),))
        self.delta_minus[self.sol_bool_infeas] = sol_elem_infeas - np.floor(sol_elem_infeas)
        self.delta_plus[self.sol_bool_infeas] = np.ceil(sol_elem_infeas) - sol_elem_infeas
        ## 根据LP求解结果更新estimate
        if obj < MAX and sol is not None:
            cost_plus,cost_minus = solver.cost_manager.get_cost()
            score = np.minimum(self.delta_plus * cost_plus,self.delta_minus * cost_minus)
            score_sum = np.sum(score)
            self.estimate = self.obj + score_sum
        else:
            self.estimate = self.obj

    def deactivate(self):
        super().deactivate()
        ## 增加中间变量，与activate对应
        del self.sol_bool_infeas,self.sol_int_infeas,self.sol_bool_feas ## 记录每个变量是否满足整数约束
        del self.delta_minus,self.delta_plus ## 每个变量为满足整数可行性约束所应变化的大小（-、+）

BBSolver类的扩展则包括对PseudoCostManager的调用以及对全局estimate信息的管理。

In [3]:
import heapq

class BBSolverExtend(BBSolver):
    def reset_branch_type(self,branch_type):
        ## 设置branch方法
        self.brancher = Brancher(self.problem,branch_type)
        
    def reset_select_type(self,select_type):
        ## 设置select方法
        self.selector = Selector(self.problem,select_type)
        
    def __init__(self,problem,branch_type=None,select_type=None):
        ## 进行基础的初始化
        super().__init__(problem)
        ## 增加对branch和select方法的选取
        self.reset_branch_type(branch_type)
        self.reset_select_type(select_type)
        ## 引入cost_manager对pseudo cost进行管理
        self.cost_manager = PseudoCostManager(self.problem)
        ## 引入队列对各结点的estimate信息进行管理
        self.unprocessed_node_estimates = []
        heapq.heapify(self.unprocessed_node_estimates)
        
    def add_node(self,node):
        super().add_node(node)
        ## 将新结点的estimate信息加入队列中
        heapq.heappush(self.unprocessed_node_estimates,(node.estimate,node.idx))
    
    def node_solve(self,node):
        super().node_solve(node)
        ## 根据LP求解结果更新cost manager
        if node.obj < MAX and node.parent is not None:
            obj_delta = node.obj - node.parent.obj
            var_delta = node.var_delta
            var_idx = node.var_idx
            self.cost_manager.update(var_idx,var_delta,obj_delta)
            
    def create_node(self,parent_node,var_idx,var_lb,var_ub,var_sol,var_bnd):
        '''
        相比原来版本，只是将Node改为Node Extend
        '''
        new_node = NodeExtend(node_idx=self.next_node_idx,parent=parent_node,cost_manager=self.cost_manager,
                        var_idx=var_idx,var_lb=var_lb,var_ub=var_ub,var_sol=var_sol,var_bnd=var_bnd,
                        init_var_lb=self.init_vars_lb[var_idx],init_var_ub=self.init_vars_ub[var_idx])
        parent_node.childs += [new_node]
        self.add_node(new_node)
        
    def solve_root_node(self):
        '''
        相比原来版本，只是将Node改为Node Extend
        '''
        ## 生成根结点
        root_node = NodeExtend()
        self.root_node = root_node
        self.add_node(root_node)
        ## 执行简化版的process
        self.unprocessed_node_idxs.remove(root_node.idx)
        root_node.activate(self) ## 激活node，生成局部信息
        self.node_solve(root_node) ## 求解子问题
        self.root_sol = root_node.sol ## 根结点（初始问题）的LP relaxation最优解
        self.global_LB = root_node.LB ## 更新BB树的上下界信息
        status = SolveStatus.ONGOING
        if not root_node.sol_feas and self.global_LB < self.global_obj:
            self.branch(root_node)
        else:
            status = SolveStatus.OPT
        root_node.deactivate() ## 释放内存
        return status

接下来，我们引入Pseudo cost manager来对pseudo cost相关信息进行管理和更新。需要特别注意的是，除了通过变量pseudo_cost_info来管理每个变量的pseudo cost信息外，还需要引入变量global_pseudo_cost_info来计算全局pseudo cost，以在某些变量的pseudo cost缺失时使用。

In [4]:
class PseudoCostManager:
    def __init__(self,problem):
        ## 初始化
        self.num_vars = problem.nCols
        self.reset()

    def reset(self):
        self.pseudo_cost_info = (np.zeros((self.num_vars,),dtype=int),np.zeros((self.num_vars,),dtype=int),
                np.zeros((self.num_vars,),dtype=float),np.zeros((self.num_vars,),dtype=float),
                np.ones((self.num_vars,),dtype=float),np.ones((self.num_vars,),dtype=float))
        self.global_pseudo_cost_info = (0,0,0,0,1,1)

    def update(self,var_idx,var_delta,obj_delta):
        '''
        pseudo cost = (\Delta obj / \Delta x_j)的均值；在这里，每步更新逻辑通过两个累加器实现
        '''
        count_plus,count_minus,value_plus,value_minus,cost_plus,cost_minus = self.pseudo_cost_info
        count_plus_global,count_minus_global,value_plus_global,\
            value_minus_global,cost_plus_global,cost_minus_global = self.global_pseudo_cost_info

        if var_delta > 0:
            ## 更新变量维度pseudo cost
            count_plus[var_idx] += 1
            value_plus[var_idx] += obj_delta / var_delta
            cost_plus[var_idx] = value_plus[var_idx] / count_plus[var_idx]
            ## 更新全局pseudo cost
            count_plus_global += 1
            value_plus_global += obj_delta / var_delta
            cost_plus_global = value_plus_global / count_plus_global
        if var_delta < 0:
            ## 更新变量维度pseudo cost
            count_minus[var_idx] += 1
            value_minus[var_idx] += -obj_delta / var_delta
            cost_minus[var_idx] = value_minus[var_idx] / count_minus[var_idx]
            ## 更新全局pseudo cost
            count_minus_global += 1
            value_minus_global += -obj_delta / var_delta
            cost_minus_global = value_minus_global / count_minus_global

        ## 对于pseudo cost信息缺失的变量，用全局pseudo cost代替
        cost_plus[count_plus == 0] = cost_plus_global
        cost_minus[count_minus == 0] = cost_minus_global

        self.pseudo_cost_info = (count_plus,count_minus,value_plus,value_minus,cost_plus,cost_minus)
        self.global_pseudo_cost_info = (count_plus_global,count_minus_global,value_plus_global,
                                        value_minus_global,cost_plus_global,cost_minus_global)
    
    def get_cost(self):
        '''
        获取各变量的pseudo cost信息
        '''
        _,_,_,_,cost_plus,cost_minus = self.pseudo_cost_info
        return cost_plus,cost_minus

在完成了上述准备后，下面，我们开始对Brancher和Selector进行扩展。

在Brancher中，我将实现inf branching、strong branching、pseudo cost branching这三种基础方法以及它们的组合方法（reliable branching）。为了避免令整个求解框架变得过于复杂，在这里只实现了各方法的最基础的思路。在实际应用场景中，可以通过引入更多超参数对各branching方法的效果进行微调，也可以引入更复杂的控制逻辑，基于求解状态切换不同的branching方法，以提高求解效率。

In [5]:
from enum import Enum,unique
@unique
class BranchType(Enum):
    ## branch方法类型
    MOST_INF = 1 ## 选取距离整数最远的变量
    LEAST_INF = 2 ## 选取距离整数最近的变量
    PSEUDO = 3 ## 通过pseudo cost估计LP目标值，进而选取变量
    FULL_STRONG = 4 ## 通过LP求解结果选取变量
    STRONG = 5 ## 通过LP求解结果选取变量
    RELIABLE = 6 ## 结合strong branching和pseudo cost branching

class Brancher:
    def __init__(self,problem,branch_type=BranchType.PSEUDO):
        ## 确定branch方法类型
        self.branch_type = branch_type
        self.branch = self.branch_pseudo
        if self.branch_type == BranchType.MOST_INF:
            self.branch = self.branch_inf_most
        elif self.branch_type == BranchType.LEAST_INF:
            self.branch = self.branch_inf_least
        elif self.branch_type == BranchType.FULL_STRONG:
            self.branch = self.branch_full_strong
        elif self.branch_type == BranchType.STRONG:
            self.branch = self.branch_strong
        elif self.branch_type == BranchType.PSEUDO:
            self.branch = self.branch_pseudo
        elif self.branch_type == BranchType.RELIABLE:
            self.branch = self.branch_reliable
    
        ## 初始化辅助变量
        self.max_iter_strong = 100 ## strong branching中用于控制dual simplex最大迭代次数
        self.max_var_strong = 100 ## strong branching中用于控制做strong branch的变量最大个数
        self.max_look_ahead = 10 ## strong branching中，如果连续多个变量对最优解没有改进，则停止搜索
        self.max_level_strong = 10 ## strong branching中用于在strong和pseudo之间切换的阈值
        self.reliable_threshold = 5 ## reliable branching中用于在strong和pseudo之间切换的阈值

    def score_func(self,score_plus,score_minus):
        '''
        打分函数，根据上下侧得分生成综合分
        '''
        score = np.maximum(score_plus,TOL) * np.maximum(score_minus,TOL)
        return score

    def check_dual_inf(self,solver,problem):
        '''
        检查当前解的对偶可行性，用于确认所得目标值是否可用
        '''
        var_status = problem.getBasisStatus()[0]
        dsol = solver.dual_sol
        dual_inf_num = np.sum(dsol[var_status == 2] >= problem.dualTolerance) + \
                        np.sum(dsol[var_status == 3] <= -problem.dualTolerance)
        if dual_inf_num > 0:
            # print(problem.objectiveValue,dual_inf_num,
            #      np.sum(np.maximum(dsol[var_status == 2],0)),
            #      np.sum(np.minimum(dsol[var_status == 3],0))) ## for debug
            return True
        return False
    
    def score_var_strong(self,solver,node,var_idx,max_iter=None):
        '''
        对列/变量j，通过调整上下界的方式（strong branching）获取其分值
        参数max_iter用于控制每个子问题的dual simplex迭代次数
        '''
        ## 获取问题基本信息
        problem = solver.problem
        basis_status = problem.getBasisStatus() ## 取出最优基
        l,u = solver.vars_lb,solver.vars_ub
        l_j,u_j,x_j = l[var_idx],u[var_idx],node.sol[var_idx]
        u_j_new,l_j_new = math.floor(x_j),math.ceil(x_j)
        obj = node.obj
        
        ## 调整上下界求解
        objs_new = []
        for (var_lb,var_ub,var_sol,var_bnd) in [(l_j,u_j_new,x_j,u_j_new),(l_j_new,u_j,x_j,l_j_new)]:
            l[var_idx],u[var_idx] = var_lb,var_ub
            if max_iter is not None:
                problem.maxNumIteration = max_iter
            problem.dual()
            is_dual_inf = self.check_dual_inf(solver,problem) ## CyLP缺少相应接口，手动实现，效率低
            obj_new = problem.objectiveValue if problem.getStatusCode() in (0,3) else MAX
            problem.setBasisStatus(*basis_status) ## 恢复最优基
            l[var_idx],u[var_idx] = l_j,u_j ## 恢复上下界
            if max_iter is not None:
                problem.maxNumIteration = 2 ** 31 - 1
            # problem.dual() ## only for debug
            if is_dual_inf:
                ## 如果对偶不可行，则求解信息不可用，废弃
                return 0
            objs_new += [obj_new]
            ## 用obj结果更新psc信息
            ##（因为dual simplex iter的限制，这里给出的psc信息与node solve中的不完全一致，需要做check）
            if obj_new >= obj and obj_new < MAX:
                solver.cost_manager.update(var_idx,var_bnd - var_sol,obj_new - obj)
        
        ## 整理结果
        score = self.score_func(objs_new[0]-obj,objs_new[1]-obj)        
        return score

    def score_pseudo_cost(self,solver,node):
        cost_plus,cost_minus = solver.cost_manager.get_cost()
        score = self.score_func(cost_plus*node.delta_plus,cost_minus*node.delta_minus)
        score[node.sol_bool_feas] = 0
        return score

    def branch_inf_most(self,solver,node):
        '''
        选取距离整数最远的变量
        '''
        score = np.minimum(node.delta_minus,node.delta_plus)
        var_idx = node.sol_int_infeas[np.argmax(score[node.sol_int_infeas])]
        return var_idx

    def branch_inf_least(self,solver,node):
        '''
        选取距离整数最近的变量
        '''
        score = np.minimum(node.delta_minus,node.delta_plus)
        var_idx = node.sol_int_infeas[np.argmin(score[node.sol_int_infeas])]
        return var_idx
    
    def branch_pseudo(self,solver,node):
        '''
        psc branching
        '''
        ## 计算psc score
        score = self.score_pseudo_cost(solver,node)
        ## 选取score最大的变量
        var_idx = solver.int_vars_idx[np.argmax(score[solver.int_vars_idx])]
        ## err check
        if node.sol_bool_feas[var_idx]:
            print('branch err',var_idx)
            var_idx = node.sol_int_infeas[np.argmax(score[node.sol_int_infeas])]
        return var_idx

    def branch_full_strong(self,solver,node):
        '''
        full strong branching
        '''
        strong_var_idxs = node.sol_int_infeas
        best_score_strong,best_var_idx_strong = -1,strong_var_idxs[0]
        for score_idx,var_idx in enumerate(strong_var_idxs):
            ## 对每个变量调用full strong branching规则获取得分
            score_strong_tmp = self.score_var_strong(solver,node,var_idx)
            if score_strong_tmp > best_score_strong:
                best_score_strong,best_var_idx_strong = score_strong_tmp,var_idx
        var_idx = best_var_idx_strong
        return var_idx

    def branch_strong(self,solver,node):
        '''
        strong branching
        '''
        if node.level >= self.max_level_strong:
            return self.branch_pseudo(solver,node)
        ## 计算psc score
        score = self.score_pseudo_cost(solver,node)
        
        ## 选取top k个变量做strong branching
        strong_var_idxs = node.sol_int_infeas
        if len(strong_var_idxs) > self.max_var_strong:
            strong_var_idxs = strong_var_idxs[np.argpartition(score[strong_var_idxs], 
                                                              -self.max_var_strong)[-self.max_var_strong:]
                                             ]
        strong_var_idxs = strong_var_idxs[np.argsort(score[strong_var_idxs])[::-1]]
        
        var_idx = strong_var_idxs[0]
        best_score_strong,best_var_idx_strong,look_ahead_cnt = score[var_idx],var_idx,0
        for score_idx,var_idx in enumerate(strong_var_idxs):
            ## 对每个变量调用branching规则获取得分
            score_strong_tmp = self.score_var_strong(solver,node,var_idx,self.max_iter_strong)
            if score_strong_tmp > best_score_strong:
                best_score_strong,best_var_idx_strong,look_ahead_cnt = \
                    score_strong_tmp,var_idx,0
            else:
                look_ahead_cnt += 1
            if look_ahead_cnt > self.max_look_ahead:
                break
        var_idx = best_var_idx_strong
        
        ## err check
        if node.sol_bool_feas[var_idx]:
            print('branch err',var_idx)
            var_idx = node.sol_int_infeas[np.argmax(score[node.sol_int_infeas])]
        return var_idx

    def branch_reliable(self,solver,node):
        '''
        reliable branching
        '''
        ## 计算psc score
        score = self.score_pseudo_cost(solver,node)
        ## 选取score最大的变量
        var_idx = solver.int_vars_idx[np.argmax(score[solver.int_vars_idx])]
        
        ## 根据psc信息的置信度，选取top k个变量做strong branching
        count_plus,count_minus,_,_,_,_ = solver.cost_manager.pseudo_cost_info
        bool_strong = (node.sol_bool_infeas) & ((count_plus <= self.reliable_threshold) | \
                                        (count_minus <= self.reliable_threshold))
        if np.any(bool_strong):
            strong_var_idxs = np.where(bool_strong)[0]
            if len(strong_var_idxs) > self.max_var_strong:
                strong_var_idxs = strong_var_idxs[np.argpartition(score[strong_var_idxs], 
                                                                  -self.max_var_strong)[-self.max_var_strong:]
                                                 ]
            strong_var_idxs = strong_var_idxs[np.argsort(score[strong_var_idxs])[::-1]]

            ## 选取score最大的变量
            best_score_strong,best_var_idx_strong,look_ahead_cnt = score[var_idx],var_idx,0
            for score_idx,var_idx in enumerate(strong_var_idxs):
                ## 对每个变量调用branching规则获取得分
                score_strong_tmp = self.score_var_strong(solver,node,var_idx,self.max_iter_strong)
                if score_strong_tmp > best_score_strong:
                    best_score_strong,best_var_idx_strong,look_ahead_cnt = \
                        score_strong_tmp,var_idx,0
                else:
                    look_ahead_cnt += 1
                if look_ahead_cnt > self.max_look_ahead:
                    break
            var_idx = best_var_idx_strong
        
        ## err check
        if node.sol_bool_feas[var_idx]:
            print('branch err',var_idx)
            var_idx = node.sol_int_infeas[np.argmax(score[node.sol_int_infeas])]
        return var_idx

在Selector中，我将实现deep first search、best first search、best estimate search这三种基础方法以及多种它们之间的组合方法。同样，为了简化整个求解流程，在这里的实现中只考虑了少量的控制参数。

In [6]:
from enum import Enum,unique
@unique
class SelectType(Enum):
    ## select方法类型
    DFS = 1 ## deep first search，选取最大深度的子问题
    BFS = 2 ## best first search，选取目标值下界最小的子问题
    BE = 3 ## best estimate，选取estimate最小的子问题
    BFS_PLUNGE = 4 ## 结合bfs和dfs
    BE_PLUNGE = 5 ## 结合be和dfs
    BFS_DIVE = 6 ## 结合bfs和dfs，先通过dfs找可行解
    BE_DIVE = 7 ## 结合be和dfs，先通过dfs找可行解

class Selector:
    def __init__(self,problem,select_type=SelectType.BE_PLUNGE):
        ## 确定select方法类型
        self.select_type = select_type
        self.select = self.select_bfs_plunge
        if self.select_type == SelectType.DFS:
            self.select = self.select_dfs
        elif self.select_type == SelectType.BFS:
            self.select = self.select_bfs
        elif self.select_type == SelectType.BE:
            self.select = self.select_be
        elif self.select_type == SelectType.BFS_PLUNGE:
            self.select = self.select_bfs_plunge
        elif self.select_type == SelectType.BE_PLUNGE:
            self.select = self.select_be_plunge
        elif self.select_type == SelectType.BFS_DIVE:
            self.select = self.select_bfs_dive
        elif self.select_type == SelectType.BE_DIVE:
            self.select = self.select_be_dive

        ## 初始化辅助变量
        self.plunge_count = 0 ## plunging连续触发次数计数器
        self.plunge_count_max = 1 ## plunging最大连续触发次数
        self.plunge_rate = 0.3 ## 用于调整plunge_count_max的具体值

    def select_dfs(self,solver,curr_node):
        '''
        deep first search
        '''
        is_root = False ## 判断是否为根结点
        while not is_root:
            ## 检查子结点
            if len(curr_node.childs) > 0:
                childs = curr_node.childs
                if len(childs) > 1 and curr_node.branch_var_idx >= 0:
                    ## 根据当前结点的branch方向，优化遍历顺序；
                    ## TODO: 这部分计算可以前置到curr_node的子结点生成中
                    x = curr_node.branch_var_sol
                    x0 = solver.root_sol[curr_node.branch_var_idx]
                    if x > x0 and childs[0].idx < childs[-1].idx:
                        childs = childs[::-1]
                for node in childs:
                    if not node.is_processed:
                        return node.idx
            ## 如果没有未处理的子结点，沿B&B树向上一级
            curr_node = curr_node.parent
            if curr_node is None:
                is_root = True
                return -1
        return -1
        
    def plunging(self,solver,curr_node):
        '''
        plunging可以看成是简化版的dfs
        '''
        ## 更新plunge_count_max的取值
        self.plunge_count_max = max(self.plunge_count_max,int(self.plunge_rate * curr_node.level),1)
        if self.plunge_count < self.plunge_count_max:
            ## 检查子结点，与dfs相同
            if len(curr_node.childs) > 0:
                childs = curr_node.childs
                if len(childs) > 1 and curr_node.branch_var_idx >= 0:
                    ## 根据当前结点的branch方向，优化遍历顺序
                    x = curr_node.branch_var_sol
                    x0 = solver.root_sol[curr_node.branch_var_idx]
                    if x > x0 and childs[0].idx < childs[-1].idx:
                        childs = childs[::-1]
                for node in childs:
                    if not node.is_processed:
                        self.plunge_count += 1
                        return node.idx

            ## 检查sibling结点
            if curr_node.parent is not None:
                siblings = curr_node.parent.childs
                for node in siblings:
                    if (not node.is_processed) and (node.idx != curr_node.idx):
                        self.plunge_count += 1
                        return node.idx

        self.plunge_count = 0
        return -1

    def select_bfs(self,solver,curr_node):
        '''
        best first search
        '''
        ## 从队列中取出下界最优的未处理子结点
        node_idx = heapq.heappop(solver.unprocessed_node_bounds)[1]
        while (len(solver.unprocessed_node_bounds) > 0 and node_idx not in solver.unprocessed_node_idxs):
            node_idx = heapq.heappop(solver.unprocessed_node_bounds)[1]
        if node_idx not in solver.unprocessed_node_idxs:
            node_idx = -1
        return node_idx

    def select_be(self,solver,curr_node):
        '''
        best estimate search
        '''
        ## 从队列中取出estimate最优的未处理子结点
        node_idx = heapq.heappop(solver.unprocessed_node_estimates)[1]
        while (len(solver.unprocessed_node_estimates) > 0 and node_idx not in solver.unprocessed_node_idxs):
            node_idx = heapq.heappop(solver.unprocessed_node_estimates)[1]
        if node_idx not in solver.unprocessed_node_idxs:
            node_idx = -1
        return node_idx

    def select_bfs_plunge(self,solver,curr_node):
        '''
        bfs + plunging：先尝试plunge，如果不可行再做bfs
        '''
        idx = self.plunging(solver,curr_node)
        if idx < 0:
            idx = self.select_bfs(solver,curr_node)
        return idx

    def select_be_plunge(self,solver,curr_node):
        '''
        be + plunging：先尝试plunge，如果不可行再做be
        '''
        idx = self.plunging(solver,curr_node)
        if idx < 0:
            idx = self.select_be(solver,curr_node)
        return idx
    
    def select_bfs_dive(self,solver,curr_node):
        '''
        bfs + plunging + dive：先尝试dfs寻找可行解，如果不可行再做bfs_plunge
        '''
        if solver.global_obj >= MAX and solver.iter_ <= 10000:
            idx = self.select_dfs(solver,curr_node)
        else:
            idx = self.select_bfs_plunge(solver,curr_node)
        return idx
    
    def select_be_dive(self,solver,curr_node):
        '''
        be + plunging + dive：先尝试dfs寻找可行解，如果不可行再做be_plunge
        '''
        if solver.global_obj >= MAX and solver.iter_ <= 10000:
            idx = self.select_dfs(solver,curr_node)
        else:
            idx = self.select_be_plunge(solver,curr_node)
        return idx

下面，我们沿用已有的求解入口来对上述branching和node selection方法进行测试。

In [7]:
from util import *

def bb_solve(model_name,branch_type,select_type,time_limit=3600,if_disp=True):
    ## 读取问题
    s = CyClpSimplex()
    s.readMps(model2fname(model_name))
    s.logLevel = 0
    solver = BBSolverExtend(s,branch_type,select_type)
    dt,status,LB,obj = solver.solve(time_limit=time_limit,if_disp=if_disp,disp_iters=100000)
    total_node_cnt,remain_node_cnt = len(solver.nodes),len(solver.unprocessed_node_idxs)
    for node in solver.nodes:
        node.clear()
    return dt,status,LB,obj,total_node_cnt,remain_node_cnt,solver

## 读取测试案例
with open('data/test_cases_v3','r+') as f:
    lines = f.readlines()
model_names = [line.strip('\n') for line in lines]

In [8]:
#############################
## 设置B&B求解中所用的branching和node selection方法
## 读者可以尝试不同的设置来观察求解结果的差异
branch_type = BranchType.RELIABLE
select_type = SelectType.BE_PLUNGE
#############################

max_secs = 600 ## 每个问题限时10分钟
result = dict()
for model_name in model_names:
    dt,status,LB,obj,total_node_cnt,remain_node_cnt,_ = \
        bb_solve(model_name,branch_type,select_type,time_limit=max_secs,if_disp=False)
    print('{}: time={:.2f}, status={}. LB={:.3e}, obj={:.3e}. total nodes={}, remain nodes={}.'.format(
        model_name,dt,status,LB,obj,total_node_cnt,remain_node_cnt))
    result[model_name] = (dt,status,LB,obj,total_node_cnt,remain_node_cnt)

## 保存测试结果
## 对于不同的branching和node selection配置，记得修改保存的文件名，以免覆盖原来的结果
df_result = process_result_new(result,save_fname='result/ours_benchmark_v2_rel_bep.csv')
print('elapsed hours: {:.2f}.'.format(df_result['time'].sum() / 3600))

binkar10_1: time=600.00, status=SolveStatus.ONGOING. LB=6.654e+03, obj=6.747e+03. total nodes=800783, remain nodes=214379.
dano3_3: time=70.26, status=SolveStatus.OPT. LB=5.763e+02, obj=5.763e+02. total nodes=59, remain nodes=0.
dano3_5: time=601.04, status=SolveStatus.ONGOING. LB=5.762e+02, obj=5.769e+02. total nodes=879, remain nodes=239.
eil33-2: time=571.83, status=SolveStatus.OPT. LB=9.340e+02, obj=9.340e+02. total nodes=27073, remain nodes=1.
gen-ip002: time=253.15, status=SolveStatus.ONGOING. LB=-4.829e+03, obj=-4.775e+03. total nodes=1653383, remain nodes=653382.
gen-ip054: time=223.58, status=SolveStatus.ONGOING. LB=6.774e+03, obj=6.861e+03. total nodes=1310731, remain nodes=310730.
gmu-35-40: time=600.00, status=SolveStatus.ONGOING. LB=-2.407e+06, obj=1.000e+20. total nodes=902595, remain nodes=224953.
gmu-35-50: time=600.00, status=SolveStatus.ONGOING. LB=-2.608e+06, obj=1.000e+20. total nodes=772105, remain nodes=331675.
icir97_tension: time=600.00, status=SolveStatus.ONGOI

通过设置不同的branching和selection方法进行求解测试，并对结果进行比较，我们可以得到以下结论：

* branching方法比较：
    * pseudo cost方法的计算效率与most inf相差不大，但算法效率有明显提升，在时限内基本能得到更高的下界和更优的可行解。
    * full strong branching的计算效率太低，对于变量数多的场景不实用。
    * strong和reliable方法可以同时利用psc的计算效率和strong branch的算法效率，但是要取得好的效果还需要选取合适的超参数。
* selection方法比较：
    * bfs方法在优化（提高）下界上效果最好，适用于prove optimality。
    * dfs方法对于整数约束较弱的案例能够更快找到可行解，但在约束较强的案例中容易卡住，既找不到可行解，也无法优化下界。
    * be方法比bfs更容易找到可行解，比dfs找到的可行解质量高。由于可行解可以用于剪枝（prune off node），高质量可行解可以有效降低需要求解的结点数，从而提高B&B的整体效率。
    * 加上plunging后，be方法在一些测试案例上有改进（sct2找到了可行解），但在另一些测试案例上算法效果（上下界gap）变差；这可能是由于plunging花了更多时间在做算法效率低的局部dfs搜索。
    * 加上dive后，be方法可以利用dfs的能力更容易找到可行解，但是整体算法效果（上下界gap）相比be+plunging要进一步变差。

上述测试显示，虽然在引入新的branching和selection方法后，B&B算法效率有明显提升，但是当前实现仍不能保证在短时间内找到问题的整数可行解。通过简单的理论分析可知，在整数变量较多的情况下，只靠branching和selection阶段的改进很难提高找到可行解的速度（极端情况下，需要遍历大多数变量才能找到整数可行解）。因此，需要进一步引入其他求解技术。

一种思路是通过启发式算法(heuristic)来搜索可行解，具体方法包括贪心算法、对LP实数解进行rounding等；不过，这类方法与B&B求解流程相对独立，故在本系列中暂不做讨论。

另一种思路则是基于以下观察：MIP问题的定义中往往隐含很多潜在reduction，即某些变量的上下界变化能够引起其他变量的上下界变化，甚至固定其他变量取值。例如，在SOS1约束$\sum_i x_i = 1,x_i \in \{0,1\}$中，如果其中一个$x_i$被branch到1，则其他所有$x_j$的取值都必须固定为0。如果能够高效地利用这些潜在reduction，则能够有效地降低B&B流程中所需branch数量，提高求解效率。这类方法在MIP文献中通常称为domain propagation，在LP中也被称为bound tightening。在下一篇blog post/notebook中，我将在割生成(cut generation)的框架下介绍这类方法。